In [1]:
import pandas as pd
import numpy as np
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# Plotly Express


In [2]:
import plotly.express as px

In [20]:
building = '北棟客梯'
input_file = building + '.csv'
df_s = pd.read_csv(input_file, index_col = 0)  
print(df_s)

     floor  hour  week direction   total  count  week_avgWait
1        1     0     0      down   27.14      3      9.046667
2        1     0     0        up  256.20     40      6.405000
3        1     0     1      down   65.04      5     13.008000
4        1     0     1        up  236.63     41      5.771463
5        1     0     2      down   54.60      7      7.800000
...    ...   ...   ...       ...     ...    ...           ...
3285    B1    23     3        up  555.78     29     19.164828
3286    B1    23     4        up  269.57     19     14.187895
3287    B1    23     5        up  340.10     16     21.256250
3288    B1    23     6      down   21.71      1     21.710000
3289    B1    23     6        up  138.31      9     15.367778

[3289 rows x 7 columns]


In [4]:
floor = 19 #手動改建築物樓層數
floor_name = []

for f in range(1,floor+1):
    if f <= 15:
        floor_name.append(str(f))
    else:
        floor_name.append("B"+str(f-15))
print(floor_name)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', 'B1', 'B2', 'B3', 'B4']


In [5]:
weekdayCnt = []
weekendCnt = []
tolWait_weekday = []
tolWait_weekend = []
avgWait_weekday = []
avgWait_weekend = []
weekday = [0,1,2,3,4]

for i in range(24):
    weekdayCnt.append([])
    weekendCnt.append([])
    tolWait_weekday.append([])
    tolWait_weekend.append([])
    avgWait_weekday.append([])
    avgWait_weekend.append([])
    for j in range(floor):
        weekdayCnt[i].append([])
        weekendCnt[i].append([])
        tolWait_weekday[i].append([])
        tolWait_weekend[i].append([])
        avgWait_weekday[i].append([])
        avgWait_weekend[i].append([])
        for d in range(2):
            weekdayCnt[i][j].append(0)
            weekendCnt[i][j].append(0)
            tolWait_weekday[i][j].append(0)
            tolWait_weekend[i][j].append(0)
            avgWait_weekday[i][j].append(0)
            avgWait_weekend[i][j].append(0)
# print(len(tolWait_weekday),len(tolWait_weekday[0]),len(tolWait_weekday[0][0]))

24 19 2


In [6]:
for i in range(1,df_s.shape[0]+1):
    for j in range(24):
        for f in range(floor):
#             print(i,df_s['hour'][i],df_s['floor'][i],df_s['direction'][i])
#             print('compare with',j,floor_name[f])
            if df_s['week'][i] in weekday:
                if df_s['hour'][i] == j and str(df_s['floor'][i]) == floor_name[f] and df_s['direction'][i]=='up':
#                     print("week - success bind with 'up'")
#                     print("tolWait_weekday[",tolWait_weekday[j][f][0])
                    weekdayCnt[j][f][0] = weekdayCnt[j][f][0] + df_s['count'][i]
                    tolWait_weekday[j][f][0] = tolWait_weekday[j][f][0] + df_s['total'][i]
                elif df_s['hour'][i] == j and str(df_s['floor'][i]) == floor_name[f] and df_s['direction'][i]=='down':
#                     print("success bind with 'down'")
                    weekdayCnt[j][f][1] = weekdayCnt[j][f][1] + df_s['count'][i]
                    tolWait_weekday[j][f][1] = tolWait_weekday[j][f][1] + df_s['total'][i]
            else:
                if df_s['hour'][i] == j and str(df_s['floor'][i]) == floor_name[f] and df_s['direction'][i]=='up':
                    weekendCnt[j][f][0] = weekendCnt[j][f][0] + df_s['count'][i]
                    tolWait_weekend[j][f][0] = tolWait_weekend[j][f][0] + df_s['total'][i]
                elif df_s['hour'][i] == j and str(df_s['floor'][i]) == floor_name[f] and df_s['direction'][i]=='down':
                    weekendCnt[j][f][1] = weekendCnt[j][f][1] + df_s['count'][i]
                    tolWait_weekend[j][f][1] = tolWait_weekend[j][f][1] + df_s['total'][i]
                    
# print("tolWait_weekday",tolWait_weekday)

In [7]:
for i in range(24):
    for f in range(floor):
        for d in range(2):
            if tolWait_weekday[i][f][d] > 0 and weekdayCnt[i][f][d] != 0:
                avgWait_weekday[i][f][d] = tolWait_weekday[i][f][d]/weekdayCnt[i][f][d]
            if tolWait_weekend[i][f][d] > 0 and weekendCnt[i][f][d] != 0:
                avgWait_weekend[i][f][d] = tolWait_weekend[i][f][d]/weekendCnt[i][f][d]
                
# print(tolWait_weekend)
# print(weekendCnt)
# print(avgWait_weekend)

In [8]:
# creating a blank series 
weekdayCol = pd.Series([]) 
weekendCol = pd.Series([]) 

# running a for loop and asigning some values to series 
for i in range(1, df_s.shape[0]+1):
    for j in range(24):
        for f in range(floor):
            if df_s['week'][i] in weekday:
                if df_s['hour'][i] == j and str(df_s['floor'][i]) == floor_name[f] and df_s['direction'][i]=='up':
                    weekdayCol[i] = avgWait_weekday[j][f][0]
                elif df_s['hour'][i] == j and str(df_s['floor'][i]) == floor_name[f] and df_s['direction'][i]=='down':
                    weekdayCol[i] = avgWait_weekday[j][f][1]
            else:
                if df_s['hour'][i] == j and str(df_s['floor'][i]) == floor_name[f] and df_s['direction'][i]=='up':
                    weekendCol[i] = avgWait_weekend[j][f][0]
                elif df_s['hour'][i] == j and str(df_s['floor'][i]) == floor_name[f] and df_s['direction'][i]=='down':
                    weekendCol[i] = avgWait_weekend[j][f][1]
                
                
# print('avgWait_weekdayCol: \n', weekdayCol)
print('avgWait_weekendCol: \n', weekendCol)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



avgWait_weekendCol: 
 11       8.505000
12       9.260577
13       8.505000
14       9.260577
25       2.192500
          ...    
3279    20.649348
3280    20.649348
3287    19.136400
3288    21.710000
3289    19.136400
Length: 930, dtype: float64


In [9]:
# inserting new column with values of list made above
df_s_weekday = pd.read_csv(input_file,index_col = 0)
df_s_weekend = pd.read_csv(input_file,index_col = 0)
print(df_s_weekend.tail())

df_s_weekday['avgWait_weekday'] = weekdayCol
df_s_weekend['avgWait_weekend'] = weekendCol
# print(df_s_weekend.tail())

     floor  hour  week direction   total  count  week_avgWait
3285    B1    23     3        up  555.78     29     19.164828
3286    B1    23     4        up  269.57     19     14.187895
3287    B1    23     5        up  340.10     16     21.256250
3288    B1    23     6      down   21.71      1     21.710000
3289    B1    23     6        up  138.31      9     15.367778


In [10]:
#分開平假日
df_s_weekday = df_s_weekday.dropna()
df_s_weekend = df_s_weekend.dropna()

df_s_weekend

,floor,hour,week,direction,total,count,week_avgWait,avgWait_weekend
11,1,0,5,down,16.22,1,16.220000,8.505000
12,1,0,5,up,410.83,46,8.931087,9.260577
13,1,0,6,down,0.79,1,0.790000,8.505000
14,1,0,6,up,552.27,58,9.521897,9.260577
25,1,1,5,down,6.51,2,3.255000,2.192500
...,...,...,...,...,...,...,...,...
3279,B1,22,5,up,595.25,25,23.810000,20.649348
3280,B1,22,6,up,354.62,21,16.886667,20.649348
3287,B1,23,5,up,340.10,16,21.256250,19.136400
3288,B1,23,6,down,21.71,1,21.710000,21.710000


In [11]:
#delete redundenct columes
df_s_weekday = df_s_weekday.drop(columns=['week','total', 'count','week_avgWait'])
df_s_weekend = df_s_weekend.drop(columns=['week','total', 'count','week_avgWait'])
df_s_weekend

,floor,hour,direction,avgWait_weekend
11,1,0,down,8.505000
12,1,0,up,9.260577
13,1,0,down,8.505000
14,1,0,up,9.260577
25,1,1,down,2.192500
...,...,...,...,...
3279,B1,22,up,20.649348
3280,B1,22,up,20.649348
3287,B1,23,up,19.136400
3288,B1,23,down,21.710000


In [12]:
# delete duplicates
df_s_weekday = df_s_weekday.drop_duplicates(keep='first', inplace=False)
df_s_weekend = df_s_weekend.drop_duplicates(keep='first', inplace=False)
print('dropped duplicates \n', df_s_weekday)
# reset_index
df_s_weekday = df_s_weekday.reset_index(drop=True)
df_s_weekend = df_s_weekend.reset_index(drop=True)
print('reset_index\n',df_s_weekday)

dropped duplicates 
      floor  hour direction  avgWait_weekday
1        1     0      down         9.378571
2        1     0        up         9.164669
15       1     1      down         9.708333
16       1     1        up         8.071036
29       1     2        up        12.527800
...    ...   ...       ...              ...
3261    B1    21      down        49.820000
3270    B1    22      down        21.230000
3271    B1    22        up        23.209262
3281    B1    23        up        17.868889
3284    B1    23      down        28.700000

[525 rows x 4 columns]
reset_index
     floor  hour direction  avgWait_weekday
0       1     0      down         9.378571
1       1     0        up         9.164669
2       1     1      down         9.708333
3       1     1        up         8.071036
4       1     2        up        12.527800
..    ...   ...       ...              ...
520    B1    21      down        49.820000
521    B1    22      down        21.230000
522    B1    22        up  

In [21]:
output_file_weekday = building+'_weekday.csv'
output_file_weekend = building+'_weekend.csv'
df_s_weekday.to_csv(output_file_weekday)
df_s_weekend.to_csv(output_file_weekend)

plotly express 有點像是 ggplot in R，<br>
只要指定x,y,color等就能自動幫你分類，<br>
下面的圖表以南棟電梯的外部電梯呼叫資料為例，<br>
x軸為樓層、以方向分開並上色，最後以時間(小時)作為動畫影格。

In [14]:
df_s_weekday = pd.read_csv(output_file_weekday, index_col = 0)
df_s_weekend = pd.read_csv(output_file_weekend, index_col = 0)

In [23]:
# floor_list = ['B4','B3', 'B2','B1','01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11','12', '13', '14', '15']

df_s_weekday.floor.unique()

fig = px.bar(data_frame=df_s_weekday, 
             
             ## 這裡的data frame都先用 R aggregate過了，以節省運算時間
             x = "floor", y = "avgWait_weekday",
             
             ## 以方向分兩張圖，並上色
             facet_col="direction",color="direction",
             
             ## 自動生成slider
             animation_frame="hour",
             
             
             labels={"count":"avgWait_weekday",
                     "hour":"Local Time (GMT+8)"})

# X axis
fig.update_xaxes(nticks=len(floor_name), ##指定breaks的數量
                 
                 ## 手動將X的類別排序
                 type="category",
                 categoryarray=floor_name)

# Y axis
fig.update_yaxes(range=[0, 200])

# Layout
fig.update_layout(title_text='Average waiting time of Elevator Calls by Floor on weekday')

fig.show()

In [16]:
# floor_list = ['B4','B3', 'B2','B1','01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11','12', '13', '14', '15']

df_s_weekend.floor.unique()

fig = px.bar(data_frame=df_s_weekend, 
             
             ## 這裡的data frame都先用 R aggregate過了，以節省運算時間
             x = "floor", y = "avgWait_weekend",
             
             ## 以方向分兩張圖，並上色
             facet_col="direction",color="direction",
             
             ## 自動生成slider
             animation_frame="hour",
             
             
             labels={"count":"avgWait_weekend",
                     "hour":"Local Time (GMT+8)"})

# X axis
fig.update_xaxes(nticks=len(floor_name), ##指定breaks的數量
                 
                 ## 手動將X的類別排序
                 type="category",
                 categoryarray=floor_name)

# Y axis
fig.update_yaxes(range=[0, 200])

# Layout
fig.update_layout(title_text='Average waiting time of Elevator Calls by Floor on weekend')

fig.show()

In [17]:
df_s_weekday.floor.unique()
fig = px.bar(data_frame=df_s_weekday, 
             x = "hour", y = "avgWait_weekday",
             facet_col="direction",color="direction",
             animation_frame="floor",
             labels={"count":"avgWait_weekday",
                     "hour":"Local Time (GMT+8)"})
# X axis
fig.update_xaxes(nticks=len(floor_name),
                 type="category",
                 categoryarray=[0,24])

# Y axis
fig.update_yaxes(range=[0, 300])

# Layout
fig.update_layout(title_text='Average waiting time of Elevator Calls by Time')

fig.show()

In [18]:
df_s_weekend.floor.unique()

fig = px.bar(data_frame=df_s_weekend, 
             x = "hour", y = "avgWait_weekend",
             facet_col="direction",color="direction",
             animation_frame="floor",
             labels={"count":"avgWait_weekend",
                     "hour":"Local Time (GMT+8)"})
# X axis
fig.update_xaxes(nticks=len(floor_name),
                 type="category",
                 categoryarray=[0,24])

# Y axis
fig.update_yaxes(range=[0, 300])

# Layout
fig.update_layout(title_text='Average waiting time of Elevator Calls by Time')

fig.show()